<h1>Rule-based chatbot about American Air</h1>
<p>Credits: Lucie Bottin, Céline Khauv,Hadi Jamal Hamad, Julie Ngan</p>

In [1]:
# Preprocessing
import pandas as pd
import re
import nltk
import random
nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from nltk.tokenize import word_tokenize, sent_tokenize
import emoji
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words=set(stopwords.words('english'))

# Doc2Vec model
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package punkt to /home/julie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/julie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/julie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/julie/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

In [3]:
def clean_text(text):    
    emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth      
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
    )"""
    
    # Remove stopwords 
    answers  = []
    for i in tweet_data.responce:
        i = give_emoji_free_text(i).lower()
        answers.append(re.sub(r'\s+', ' ',
                            re.sub(r'\[[0-9]*\]', ' ',
                            re.sub(r'(@([A-Za-z0-9`~!@#$%^&*()_|+\-=?;:\'",.<>\{\}\[\]\\\/]{2,32}))', '', # remove tags
                            re.sub(r'\d+', ' ', # remove numbers
                            re.sub(r'\#+', ' ', # remove hashtags
                            re.sub(r'http\S+', ' ', # remove urls
                            re.sub(emoticon_string,'', # remove emojis
                            re.sub(r'[\.\?\!\,\:\;\"]', '', i) # remove punctuation
                                  ))))))))
    # Lemmatization and tokenization
    sentences = []
    le = WordNetLemmatizer()
    for i in answers:
        word_tokens = word_tokenize(i)
        lemmas = [ le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w) > 3 ]
        cleaned_text=" ".join(lemmas)
        sentences.append(cleaned_text)
    

    
    return sentences

In [4]:
# Retrieve the questions and answers of Twitter
tweet_data = pd.read_csv("question_responce.csv")
tweet_data = tweet_data[["question","responce"]]

In [5]:
cleaned_sentences = clean_text(tweet_data.responce)

/tmp/ipykernel_33539/4218898814.py:2: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(r'', text)


<h2>Tag and train data with Doc2Vec</h2>

In [6]:
processed_corpus = preprocess_documents(cleaned_sentences)
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(cleaned_sentences)]

In [7]:
tagged_data

[TaggedDocument(words='sure pas along kind word aateam', tags=[0]),
 TaggedDocument(words='apology delay responding made know still need assistance', tags=[1]),
 TaggedDocument(words='definitely future pilot making happyhalloween', tags=[2]),
 TaggedDocument(words='sorry frustration', tags=[3]),
 TaggedDocument(words='glad kick back enjoy show flying thanks kind word', tags=[4]),
 TaggedDocument(words='never want experience anything le perfect sorry wi-fi slower expected', tags=[5]),
 TaggedDocument(words='thanks shout-out stopping marc pas admiral club manager', tags=[6]),
 TaggedDocument(words='happy walk help booking process', tags=[7]),
 TaggedDocument(words='sorry long wait next available agent soon possible', tags=[8]),
 TaggedDocument(words='fare sell real time sorry able snag lower fare', tags=[9]),
 TaggedDocument(words='glad reservation team able help', tags=[10]),
 TaggedDocument(words='sorry frustration damage must assessed person within hour arrival airport personnel', tag

In [ ]:
model = Doc2Vec(tagged_data, 
                vector_size=200, 
                epochs=30, 
                hs=1, # use of hierachical softmax
                min_count=1, 
                window=2, # Max distance
                dm =0) # PV-Bag of words used
  
model.build_vocab(tagged_data)
train_corpus = [taggeddoc for taggeddoc_list in tagged_data for taggeddoc in taggeddoc_list]

for epoch in range(model.epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=2,
                epochs=100)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0


In [ ]:
def response(user_query):
    #Add the query to the answers list and vectorize it, then measure the cosine similarity between query and answers
    cleaned_sentences.append(user_query)
    vectorizer = TfidfVectorizer(stop_words=stop_words,max_features=1000)
    sentences_vectors = vectorizer.fit_transform(cleaned_sentences)
    
    similar_vector_values = cosine_similarity(sentences_vectors[-1], sentences_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    matched_vector = matched_vector[-2]

    if matched_vector == 0: # no vector matching
        return "I didn't understand, can you reformulate please ?"
    else: 
        return cleaned_sentences[similar_sentence_number]

In [ ]:
# Greetings
greetings = ('hey', 'hello', 'whats up', 'good morning', 'good evening', 'morning', 'evening', 'hello there', 'hey there')
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]
# Goodbye
goodbye = ('bye', 'good bye', 'take care')
goodbye_responses = ['See you next time !', 'Bye see you next time ! Hope I was helpful :p']
# Apology
apology = ['sorry', 'I apologize', 'disapointed']
# Thanks
thanks =('thank you', 'thanks', 'grateful', 'pleased')
thanks_responses = ['Always a pleasure to help you !', 'You\'re welcome !', 'No problem ! :)']
def greeting_response(query):
    for token in query.split():
        if token.lower() in greetings:
            return random.choice(greeting_responses)
        if token.lower() in thanks:
            return random.choice(thanks_responses)
        if token.lower() in goodbye:
            return random.choice(goodbye_responses)
            break

In [ ]:
print("Hello, I am the bot assistant of American Air, here to answer to your questions. Please ask me anything :))")

while(True):
    query = input().lower()
    if query not in goodbye:
        if query in greetings or query in thanks or query in goodbye:
            print(greeting_response(query))
        else:
            print("Twitter Chatbot: ", end="")
            print(response(query))
            cleaned_sentences.remove(query)
    else:
        print()
        break